# 0. 환경설정
이번 ex 에서는 rnn모델에 시퀀스가 있는 데이터를 학습시켜서 문장 출력기로 작동할 수 있도록 합니다. 작사가 인공지능을 만들기위해   
각 가수별 txt파일을 만들어줍니다. 이 파일은 가사들의 모음집입니다.   
raw_corpus라는 리스트를 생성한 후 파일 안에 있는 텍스트들을 읽어 리스트 안에 나열해줍니다.   
우르는 이제 이 raw_corpus로 전처리를 진행할겁니다.

In [1]:
import glob
import os
import re
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' #파일을 불러옵니다

txt_list = glob.glob(txt_file_path)

raw_corpus = []  #raw_corpus라는 리스트 생성 후 txt파일을 읽어서 리스트 안에 담습니다

for txt_file in txt_list:              #파일 안에 있는 
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:10])

데이터 크기: 187088
Examples:
 ['\ufeffThey call me Mr Carter I kissed the daughter', 'Of the deads forehead I killed the father', 'Spilled the heart of a mildew hater', 'I will put them body on chill like glaciers', 'Gracias Im crazy yes its obvious', 'Going against me is atheist', 'I got my angels on my shoulders and a quarter of that angel dust', 'I aint sniffin Im just pitchin ya honor I aint snitchin ya honor', 'Hate bitch niXgas bitches with power', 'Vacate when the kitchen get hotter']


# 1. 데이터 정제

데이터 전처리 과정을 진행합니다. 
**preprocess_sentence 라는 함수를 정의해서 필요 없는 내용들을 제거해줍니다.**    
<start>와 <end>라는 특수한 토큰도 추가하여 언어모델이 학습하기 편하도록 가공해줍니다.

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()   # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)      # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)             # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'  # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [3]:
#corpus라는 빈 리스트를 생성하여 위에서 정의한 함수를 실행해 차례로 넣어줍니다. 
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> they call me mr carter i kissed the daughter <end>',
 '<start> of the deads forehead i killed the father <end>',
 '<start> spilled the heart of a mildew hater <end>',
 '<start> i will put them body on chill like glaciers <end>',
 '<start> gracias im crazy yes its obvious <end>',
 '<start> going against me is atheist <end>',
 '<start> i got my angels on my shoulders and a quarter of that angel dust <end>',
 '<start> i aint sniffin im just pitchin ya honor i aint snitchin ya honor <end>',
 '<start> hate bitch nixgas bitches with power <end>',
 '<start> vacate when the kitchen get hotter <end>']

In [4]:
#벡터화
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.
    
    a = []
    for i in tensor:
        if len(i) < 16:
            a.append(i)
    

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(a, padding='post', maxlen=15, truncating='post')
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   45  154 ...    0    0    0]
 [   2   19    6 ...    0    0    0]
 [   2 4811    6 ...    0    0    0]
 ...
 [   2   75   46 ...    3    0    0]
 [   2   49    4 ...    0    0    0]
 [   2   13  635 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f0fd0f8f450>


 ```maxlen```의 디폴트값이 None이기 때문에 maxlen=15에 맞춰 시퀀스를 잘라줍니다.     
 ```truncating='post'```설정해주면 뒷 부분의 패딩을 길이 15에 맞춰 잘라주지 않을까했는데 우리 눈에 보이는 데이터 값만 줄어들었지
 실질  데이터는 줄어들지 않았습니다.

import pandas as pd #판다스를 불러와서 데이터를 살펴볼 수 있을까요. 이미 토큰화된 데이터라 의미가 없을까요.
se = tensor
raw_corpus.head

tensor도 raw_corpus도 list라 head를 생성할 수 없다고 뜨네요.그럼 데이터를 어떻게 뜯어보고 문장 길이가 15 이상인 데이터를 지울 수 있을까요

```python
a = []
    for i in tensor:
        if len(i) < 16:
            a.append(i)
```
       
이렇게 a라는 리스트를 생성해 준 후 for문을 사용하여 tensor중 전체 길이가 15이하인 자료들만 모아 a안에 나열해줍니다. 

In [6]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   2   45  154   12  581 2474    5 1327    6  983    3    0    0    0]
[  45  154   12  581 2474    5 1327    6  983    3    0    0    0    0]


# 2. 평가 데이터셋 분리
훈련 데이터와 평가 데이터를 분리하세요!     

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상으로 설정하세요! 총 데이터의 20%를 평가 데이터셋으로 사용해 주세요!     

이 때  validation data도 설정해줍니다

In [17]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input, 
                                                          test_size=0.2,
                                                          shuffle=True)
print(src_input)



[[   2   45  154 ...    0    0    0]
 [   2   19    6 ...    0    0    0]
 [   2 4811    6 ...    0    0    0]
 ...
 [   2   75   46 ...  879    3    0]
 [   2   49    4 ...    0    0    0]
 [   2   13  635 ...    0    0    0]]


In [18]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


In [9]:
#validation_set 도 한번 확인해봅니다. 잘 나누어졌네요.
print(enc_val.shape)
print(dec_val.shape)

(31203, 14)
(31203, 14)


In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# 3. 인공지능 만들기

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 50
hidden_size = 500
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [12]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-8.71564480e-05, -1.99784081e-05, -2.45842621e-05, ...,
         -2.90441367e-05,  8.87276401e-05, -1.09521125e-06],
        [-9.60817488e-05,  2.17306042e-05, -9.61165642e-05, ...,
         -3.03258803e-05,  1.66635626e-04,  1.19243639e-04],
        [-2.23834995e-05,  9.73667629e-05, -1.57360235e-04, ...,
         -6.23051455e-05,  1.79317823e-04,  2.22541814e-04],
        ...,
        [ 6.92879257e-04,  4.13573260e-04, -5.13440231e-04, ...,
         -1.68029583e-04, -6.57878481e-05,  3.50400485e-04],
        [ 8.17462220e-04,  5.02296258e-04, -5.40322391e-04, ...,
         -1.48559076e-04, -9.83860737e-05,  3.72344191e-04],
        [ 9.25616245e-04,  5.88129915e-04, -5.59980283e-04, ...,
         -1.28669140e-04, -1.22311627e-04,  3.97416647e-04]],

       [[-8.71564480e-05, -1.99784081e-05, -2.45842621e-05, ...,
         -2.90441367e-05,  8.87276401e-05, -1.09521125e-06],
        [-2.17519206e-04, -1.14054630e-04,  5

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  600050    
_________________________________________________________________
lstm (LSTM)                  multiple                  1102000   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  2002000   
_________________________________________________________________
dense (Dense)                multiple                  6012501   
Total params: 9,716,551
Trainable params: 9,716,551
Non-trainable params: 0
_________________________________________________________________


In [14]:
#model을 학습시킵니다. validation loss도 확인해줍니다. 원하는 대로 2.2 이하로 나오네요
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30, verbose=1, validation_data=(enc_val, dec_val))
                    


Epoch 1/30
609/609 [==============================] - 36s 60ms/step - loss: 3.7054 - val_loss: 3.2651
Epoch 2/30
609/609 [==============================] - 36s 59ms/step - loss: 3.2040 - val_loss: 3.1085
Epoch 3/30
609/609 [==============================] - 35s 57ms/step - loss: 3.0631 - val_loss: 2.9722
Epoch 4/30
609/609 [==============================] - 35s 58ms/step - loss: 2.9493 - val_loss: 2.8718
Epoch 5/30
609/609 [==============================] - 34s 55ms/step - loss: 2.8593 - val_loss: 2.7895
Epoch 6/30
609/609 [==============================] - 34s 55ms/step - loss: 2.7846 - val_loss: 2.7198
Epoch 7/30
609/609 [==============================] - 34s 55ms/step - loss: 2.7197 - val_loss: 2.6594
Epoch 8/30
609/609 [==============================] - 34s 55ms/step - loss: 2.6605 - val_loss: 2.5996
Epoch 9/30
609/609 [==============================] - 34s 55ms/step - loss: 2.6046 - val_loss: 2.5480
Epoch 10/30
609/609 [==============================] - 34s 55ms/step - loss: 2.551

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [16]:
#문장을 생성해봅니다.
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he s a monster <end> '

# 후기
1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?
he's a monster 이라는 문장을 뽑아냈다. 다만 '가 전처리 중에 사라진 게 아쉽다.
2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?
-토크나이저 진행
-총 데이터의 0.2를 평가 데이터셋으로 지정했다. 진행사항 중 enc_train, enc_val, dec_train, dec_val을 확인해볼 수 있다.
-문장의 시퀀스 길이를 15로 맞춰주었고 길이가 긴 데이터는 삭제해주었다. 패딩은 Post로 넣어주었다.
3. 텍스트 생성모델이 안정적으로 학습되었는가?
val_loss: 1.8817를 얻었다.


쉽지는 않았지만 결과물을 상상하면 흥미로웠다. rnn모델이 자체적으로 다음에 올 단어를 예측해서 문장을 뽑아내는게 신기하다.     
GPT-2는 얼마나 많은 양의 텍스트를 학습한걸까. 